### Вариант 3

Провести морфологический анализ текста с простейшим снятием омонимии. Организовать хранение результатов разбора в префиксном дереве, при этом в качестве первого ключа используется часть речи, а в качестве следующих — грамматические параметры слов. Слова с совпадающей частью речи и грамматическими параметрами хранить в отсортированном списке.

In [1]:
import pymorphy2 as pm2
from re import sub

morph = pm2.MorphAnalyzer()

In [2]:
def cleaning(text): 
    
    cleantext = ''
    with open(text,'r',encoding='utf-8') as f:
    # Чистим текст от знаков препинания и некириллических символов 
        words = f.read()
        for i in words:
            if i in '''ЙЦУКЕНГШЩЗХЪЭЖДЛОРПАВЫФЯЧСМИТЬБЮЁ
                    йцукенгшщзхъэждлорпавыфячсмитьбюё
                    1234567890- ''':
                cleantext += i
    # Заменяем пробелоподобные символы на пробелы
    cleantext = sub(r'\s',' ',cleantext)
    cleantext = cleantext.split()
   
    return cleantext

In [3]:
def parsing(text):
    
    an = [] # общий список разборов
    for word in text:
        scores = []
        # Парсим слово с помощью pymorphy2
        p = morph.parse(word)
        # Снимаем омонимию
        # Выбираем самую вероятную трактовку благодаря параметру score
        scores.append(i.score for i in p)
        n = scores.index(max(scores)) 
        p = morph.parse(word)[n]
        # Добавляем самый вероятный вариант разбора в общий список разборов всех слов
        an.append(p.tag)
    
    return an

In [4]:
def prepareanalysis(an): 
    # Для каждого слова анализ сейчас выглядит примерно так: 'VERB,impf,tran plur,3per,pres,indc'
    # нам надо сделать из этого список
    
    listofgr = [] 
    for gr in an:
        gr = str(gr)
        gr = gr.split(',') # делим элементы по запятой, однако и сейчас у нас есть значения вроде [tran plur]
                           # мы хотим поделить их на 2 отдельных элемента
        for i in range(len(gr)):
            gr[i] = gr[i].split() # у нас оказались вложенные списки
        listofgr.append([i for sublist in gr for i in sublist]) # делаем одномерный список
                                                                # теперь каждый анализ будет выглядеть примерно так:
                                                                # ['VERB','impf','tran','plur','3per','pres','indc']
    
    return listofgr

In [5]:
def addPostfix(postfix, curLevel):
    
    # Если в префиксе остался только один элемент, то это наше слово
    if len(postfix) == 1:  
        # Добавим словарь {'слова':[слово 1, слово 2, слово 3...]} с элеменатми с одинаковыми признаками в дерево
        if curLevel.get('слова', None) == None:
            curLevel.update({'слова': [postfix[0]]})
        else:
            # Не будем добавлять одно слово много раз, чтобы не загружать дерево
            if postfix[0] not in curLevel['слова']:
                curLevel['слова'].append(postfix[0])
                curLevel['слова'] = sorted(curLevel['слова'])
        return
    # Такого начала нет в словаре. Его надо добавить.
    if curLevel.get(postfix[0], None) == None:
        curLevel[postfix[0]] = {}
    # Добавляем оставшуюся часть в соответствующий узел.
    addPostfix(postfix[1:], curLevel[postfix[0]])

In [14]:
text = 'input.txt'
words = cleaning(text)
analysis = parsing(words)
properanalysis = prepareanalysis(analysis)
trie = {}
for i in range(len(properanalysis)):
    properanalysis[i].append(words[i].lower()) # добавляем в конец каждого списка само слово, чтобы внести его в дерево
    addPostfix(properanalysis[i], trie)

#print(trie)
#print(trie.keys())